In [3]:
import os

# Changing the current working directory to its parent directory using ".."
# os.chdir("..")

# Add .. to python path
import sys
sys.path.append("..")

In [4]:
import pandas as pd
from config import PATH_TO_SAVE
models_info = pd.read_csv(os.path.join(PATH_TO_SAVE, "models_info.csv"))

ModuleNotFoundError: No module named 'config'

In [15]:
from datetime import datetime

def get_time_position(date, model_last_date):
    # Count the number of times between the last date and the date we want (by 10 minutes)
    return (date - model_last_date).days * 144 + (date - model_last_date).seconds // 600

date = datetime(2020, 4, 5, 23, 50)
get_time_position(date, MODEL_LAST_DATE)

864

In [16]:
import torch
from darts.models import TCNModel

# Load the model
model = TCNModel.load("models/tcn_model_20230316-141730.pt")

# Make predictions
output = model.predict(864)



FileNotFoundError: [Errno 2] No such file or directory: 'models/tcn_model_20230316-141730.pt'

In [ ]:
output["ActivePower"].pd_dataframe()

component,ActivePower
ds,
2020-03-31 00:00:00,-3.667194e+02
2020-03-31 00:10:00,-1.441123e+02
2020-03-31 00:20:00,6.252472e+02
2020-03-31 00:30:00,1.736000e+03
2020-03-31 00:40:00,2.983776e+03
...,...
2020-04-05 23:10:00,6.979904e+296
2020-04-05 23:20:00,1.544192e+297
2020-04-05 23:30:00,3.416277e+297
